# VQQ Lab


**Author: Duc Hoang,  Philip Harris (MIT)**

**Date: 9/1/2022**

---

In [ ]:
# First let's import what we need for our lab first
# uproot High energy physics python file format => https://masonproffitt.github.io/uproot-tutorial/aio.html
import uproot
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from scipy.optimize import curve_fit
import scipy.stats as stats

#plotting style for High Energy physics 
import lmfit as lm
import mplhep as hep
plt.style.use(hep.style.CMS)

### Loading the data

Here is some codes to load the correct data sets!

In [ ]:
# Now lets look at the data. Our data sample is the JetHT dataset. 
# What that means is the data passed triggers that have a jet in one of the triggers. (discuss below)
data   = uproot.open("data/JetHT_s.root")["Tree"]

# In addition to above we have Monte Carlo Simulation of many processes
# Some of these process are well modelled in simulation and some of them are not
#-------------------------------------------------------------------------------

# Now we have our actual process qq=>W=>qq at 8TeV collision energy
wqq    = uproot.open("data/WQQ_s.root")["Tree"]

# Now we have our actual process qq=>Z=>qq at 8TeV collision energy
zqq    = uproot.open("data/ZQQ_s.root")["Tree"] 

#Hint: You could check for files in the data directory by doing "!ls data/" in a cell.
#The ZQQ file name is similar to WQQ

# Unfortunately the samples I made above a long time ago are very small. 
# To train NNs and make nice plots we will use larger samples produced at a different collision energy
# qq=>W=>qq at 13TeV collision energy
wqq13  = uproot.open("data/skimh/WQQ_sh.root")["Tree"]

# qq=>Z=>qq at 13TeV collision energy
zqq13  = uproot.open("data/skimh/ZQQ_sh.root")["Tree"]

# Now we have our worst modeled background this is also our main background. 
# This is is our di-jet quark and gluon background. 
# We just call these backgrounds QCD because they are produced with Quantum Chromo Dynamics. 
qcd    = uproot.open("data/QCD_s.root")["Tree"]

# Now we have the Higgs boson sample (we might need this in the future)
ggh    = uproot.open("data/ggH.root")["Tree"]

# And top-quark pair production background. 
tt     = uproot.open("data/TT.root")["Tree"]

# Finally we have the rarer double W, W+Z and Z+Z diboson samples where we have two bosons instead of one
ww     = uproot.open("data/WW.root")["Tree"]
wz     = uproot.open("data/WZ.root")["Tree"]
zz     = uproot.open("data/ZZ.root")["Tree"]

## Simple helper functions

After loading the data, you are provided with some simple helper functions that have already been used in our introduction notebook. These are used for pre-selection (standard cuts that physicsists usually apply before making measurements) and computing the scaling factor of datasets. You can review the previous notebook for further explanations!

In [ ]:
def selection(iData):
    '''
    Standard pre-selection
    '''
    #lets apply a trigger selection
    trigger = (iData.arrays('trigger', library="np")["trigger"].flatten() > 0)

    #Now lets require the jet pt to be above a threshold (400 TODO: ASK about units)
    jetpt   = (iData.arrays('vjet0_pt', library="np")["vjet0_pt"].flatten() > 400)

    #Lets apply both jetpt and trigger at the same time
    #standard_trig = (iData.arrays('trigger', library="np")["trigger"].flatten() % 4 > 1) #lets require one of our standard triggers (jet pT > 370 )
    allcuts = np.logical_and.reduce([trigger,jetpt])

    return allcuts
    
def get_weights(iData,weights,sel):
    
    weight = weights[0]
    
    for i in range(1,len(weights)):
        weight *= iData.arrays(weights[i],library="np")[weights[i]][sel]
        
    return weight

def integral(iData,iWeights):
    '''
    This computs the integral of weighted events 
    assuming a selection given by the function selection (see below)
    '''
    
    #perform a selection on the data (
    mask_sel=selection(iData)
    
    #now iterate over the weights not the weights are in the format of [number,variable name 1, variable name 2,...]
    weight  =iWeights[0]
    
    for i0 in range(1,len(iWeights)):
        weightarr = iData.arrays(iWeights[i0], library="np")[iWeights[i0]][mask_sel].flatten()
        
        #multiply the weights
        weight    = weight*weightarr
    
    #now take the integral and return it
    return np.sum(weight)


def scale(iData8TeV,iData13TeV,iWeights):
    '''
    This computes the integral of two selections for two datasets labelled 8TeV and 13TeV,
    but really can be 1 and 2. Then it returns the ratio of the integrals
    '''
    
    int_8TeV  = integral(iData8TeV,iWeights)
    int_13TeV = integral(iData13TeV,iWeights)
    
    return int_8TeV/int_13TeV

-----

# 0. Find the W Peak

In this section, we provide you with some example codes to investigate the W peak. In the next section, we will ask you to fit it. First, let's make a plot of the data without any cuts. 

In [ ]:
def plotDataSim(iVar, iSelection, iVarName, iRange):
    
    #Lets Look at the mass
    weights = [1000*18300, "puweight", "scale1fb"]
    mrange = iRange #range for mass histogram [GeV]
    bins=40            #bins for mass histogram
    density = False     #to plot the histograms as a density (integral=1)

    qcdsel      = iSelection(qcd)
    wsel        = iSelection(wqq13)
    zsel        = iSelection(zqq13)
    datasel     = iSelection(data)
    ttsel       = iSelection(tt)
    wwsel       = iSelection(ww)
    wzsel       = iSelection(wz)
    zzsel       = iSelection(zz)
    gghsel      = iSelection(ggh)

    wscale=scale(wqq,wqq13,weights)
    zscale=scale(zqq,zqq13,weights)

    # Getting the masses of selected events
    dataW = data.arrays(iVar, library="np") [iVar][datasel]
    qcdW  = qcd.arrays(iVar, library="np")  [iVar][qcdsel]
    wW    = wqq13.arrays(iVar, library="np")[iVar][wsel]
    zW    = zqq13.arrays(iVar, library="np")[iVar][zsel]
    zzW   = zz   .arrays(iVar, library="np")[iVar][zzsel]
    wzW   = wz   .arrays(iVar, library="np")[iVar][wzsel]
    wwW   = ww   .arrays(iVar, library="np")[iVar][wwsel]
    ttW   = tt   .arrays(iVar, library="np")[iVar][ttsel]
    gghW  = ggh  .arrays(iVar, library="np")[iVar][gghsel]

    #Define the weights for the histograms
    hist_weights = [get_weights(qcd,weights,qcdsel),
                    get_weights(wqq13,weights,wsel)*wscale,
                    get_weights(zqq13,weights,zsel)*zscale,
                    get_weights(zz,weights,zzsel),
                    get_weights(wz,weights,wzsel),
                    get_weights(ww,weights,wwsel),
                    get_weights(tt,weights,ttsel),
                   ]

    #Hint: Provide a list of selected data
    plt.hist([qcdW,wW, zW, zzW, wzW, wwW, ttW],
             color=["royalblue","r", "orange","g", "b", "purple", "cyan",], 
             label=["QCD", "W", "Z", "ZZ", "WZ", "WW", "tt",], 
             weights=hist_weights,
             range=mrange, bins=50, alpha=.6, density=density,stacked=True)

    #Other configurations for the histogram
    counts, bins = np.histogram(dataW, bins=bins, range=mrange, density=density)
    yerr = np.sqrt(counts) / np.sqrt(len(dataW)*np.diff(bins))
    binCenters = (bins[1:]+bins[:-1])*.5
    plt.errorbar(binCenters, counts, yerr=yerr,fmt="o",c="k",label="data")
    plt.legend()
    plt.xlabel(iVarName)
    plt.ylabel("Counts")
    plt.show()

plotDataSim("vjet0_msd0", selection, "Jet Mass", [40,200])
plotDataSim("vjet0_t2", selection, r"$\tau_2$", [0,0.5]) 
#Add some code here to compare variables

When we want to compare variables, we plot the variable, the optimal cut is often at the point where the signal and background cross. As a example look at the plot below, we see that the $\tau_2/\tau_1$ point cuts at around 0.5. 

In [ ]:
def plot_tau21():
    
    #Lets Look at the mass
    weights = [1000*18300, "puweight", "scale1fb"]

    qcdsel      = selection(qcd)
    wsel        = selection(wqq13)

    wscale=scale(wqq,wqq13,weights)

    # Getting the masses of selected events
    qcd_t21  = (qcd.arrays('vjet0_t2', library="np")['vjet0_t2'][qcdsel]/
               qcd.arrays('vjet0_t1', library="np")['vjet0_t1'][qcdsel])
    
    w_t21    = (wqq13.arrays('vjet0_t2', library="np")['vjet0_t2'][wsel]/
               wqq13.arrays('vjet0_t1', library="np")['vjet0_t1'][wsel])

    plt.hist(qcd_t21,
             weights=get_weights(qcd,weights,qcdsel),
             bins=50,
             color='red',
             label="QCD", alpha=.6, density = True)
    
    plt.hist(w_t21,
             weights=get_weights(wqq13,weights,wsel)*wscale,
             bins=50,
             color='black',
             label="W", alpha=.6, density = True)

    #Other configurations for the histogram
    plt.legend()
    plt.xlabel(r"$\tau_{2}/\tau_{1}$")
    plt.ylabel("Counts")
    plt.show()

plot_tau21()

In [ ]:
def selectionW(iData):
    '''
    This is the specific selection for selecting out events with W signal for our analysis
    '''
    
    #Pre-selection citeria
    trigger = (iData.arrays('trigger', library="np")["trigger"].flatten() >= 0)
    jetpt   = (iData.arrays('vjet0_pt', library="np")["vjet0_pt"].flatten() >= 400)
    
    #Select the jets to compute tau2/tau1
    jett2   = (iData.arrays('vjet0_t2', library="np")["vjet0_t2"].flatten())
    jett1   = (iData.arrays('vjet0_t1', library="np")["vjet0_t1"].flatten())
    
    t21 = jett2/jett1
                                
    #And then perform the cut
    #Hint: You could determine the threshold of the cut by plotting the distribution of 
    #t21ddt scores for W and background and then determine a ball park threshold
    #where you think the W signal would be best selected
    #Or more simply you could look at the given plot and determine the appropriate threshold.
    
    t21cut   = t21 < 0.5
    
    allcuts = np.logical_and.reduce([trigger, jetpt, t21cut])
    
    return allcuts

In [ ]:
plotDataSim("vjet0_msd0", selectionW, "Jet Mass",[40,200])

Looking at this plot, we can see that the W peak is not at all obvious to find! This is why we need to employ additional techniques in order to clearly identify the W peak, which is what you'll have the chance to do in the next section!

# 1. Fit for W Peak

Now, lets do the lab. <font color="blue"> Your first challenge is to make a mass plot and perform fitting for W signal. </font> A hint is that the plots should come out similarly to this (it doesn't have to be exactly the same), where on the left we show the soft-drop mass/groomed mass ($m_{SD}$) distribution for different processes in the Monte Carlo simulation along with the real data. The plot on the right shows the fit:

<img src="images/S50_WFit.png" width='900'>

-----

## 1.1 W Mass Plot

Since finding W peak is hard, we need to use another parameter, $\rho$, which is a scaling variable for QCD jets. This parameter adds another channels of mass and $p_T$ to our selection, helping us to refine our W peak.

$\rho$ is defined in this paper:

https://arxiv.org/pdf/1603.00027.pdf

Your first goal is to figure out how $\rho$ is defined by quoting the paper, and then figure out the best selections based on a combination of $\rho$ and $\tau_2/\tau_1$. The final cut is based on a parameter defined as *DT* (Deccorelated Taggers) score:

$$(\tau_2/\tau_1)_{dt} = \tau_2/\tau_1 - (\text{your correlation})*\rho$$

Where the correlation **<em> your correlation </em>** is the correlation coefficient between $\tau_2/\tau_1$ and $\rho$. 

To figure out the correlation, let's plot $\tau_2/\tau_1$ and $\rho$ in the data first!

In [ ]:
def plot_taus_and_rho(iData):
    
    jetptnocut = (iData.arrays('vjet0_pt', library="np")["vjet0_pt"].flatten())
    jetmass = (iData.arrays('vjet0_msd0', library="np")["vjet0_msd0"].flatten())
    jett2   = (iData.arrays('vjet0_t2', library="np")["vjet0_t2"].flatten())
    jett1   = (iData.arrays('vjet0_t1', library="np")["vjet0_t1"].flatten())
    
    #Define rho according to the paper
    rho = # Definition from the paper
    
    #Define tau2/tau1
    t21 = # The ratio
    
    plt.hist2d(rho, t21, bins = 40)
    
    plt.xlabel(r"$\rho$")
    plt.ylabel(r"$\tau_2/\tau_1$")
    plt.show()
    
plot_taus_and_rho(qcd)

-----

Great! Now we can fit a line on the 2D histogram to determine the correlation! Here we give you the fitting code. The codes fit by putting a threshold on the 2D histogram to selectively fit on the most relevant data points, your task for this is to play around with the threshold to determine the best fit!

In [ ]:
def fit_correlation(iData):
    
    jetptnocut = (iData.arrays('vjet0_pt', library="np")["vjet0_pt"].flatten())
    jetmass = (iData.arrays('vjet0_msd0', library="np")["vjet0_msd0"].flatten())
    jett2   = (iData.arrays('vjet0_t2', library="np")["vjet0_t2"].flatten())
    jett1   = (iData.arrays('vjet0_t1', library="np")["vjet0_t1"].flatten())
    
    #Define rho according to the paper
    rho = np.log(jetmass**2/jetptnocut)
    
    #Define tau2/tau1
    t21 = jett2/jett1
    
    plt.hist2d(rho, t21, bins = 40)
    plt.xlabel(r"$\rho$")
    plt.ylabel(r"$\tau_2/\tau_1$")
    
    #Fit the line
    #Produce 2D histogram
    H,xedges,yedges = np.histogram2d(rho,t21, bins=40,density = True)
    
    bin_centers_x = (xedges[:-1]+xedges[1:])/2.0
    bin_centers_y = (yedges[:-1]+yedges[1:])/2.0
    
    #Find the non-zero indicies
    non_zero_idx = np.argwhere(H > 0.6) #You can play around with this!
    x_idx = non_zero_idx[:,0]
    y_idx = non_zero_idx[:,1]
    
    x_coord = [bin_centers_x[x_idx[i]] for i in range(0,len(x_idx))]
    y_coord = [bin_centers_y[y_idx[i]] for i in range(0,len(y_idx))]
    
    #Fit a linear model on the points plotted
    def func(x, a, b):
        return a * x + b
    plt.scatter(x_coord, y_coord)
    
    popt, pcov = curve_fit(func, x_coord, y_coord)
    plt.plot(bin_centers_x, func(bin_centers_x, *popt), 'r-',
             label='fit: a=%5.3f, b=%5.3f' % tuple(popt))
    
    #Show the fit result
    legend = plt.legend()
    plt.setp(legend.get_texts(), color='w')
    plt.show()

fit_correlation(qcd)

Now use the correlation to define $(\tau_2/\tau_1)_{dt}$ and plot it with $\rho$ to verify that we have successfully decorrelate the tagger! (you should see a straight-line distribution in the histogram).

In [ ]:
def plot_tausdt_and_rho(iData):
    
    jetptnocut = (iData.arrays('vjet0_pt', library="np")["vjet0_pt"].flatten())
    jetmass = (iData.arrays('vjet0_msd0', library="np")["vjet0_msd0"].flatten())
    jett2   = (iData.arrays('vjet0_t2', library="np")["vjet0_t2"].flatten())
    jett1   = (iData.arrays('vjet0_t1', library="np")["vjet0_t1"].flatten())
    
    #Define rho according to the paper
    rho = np.log(jetmass**2/jetptnocut)
    
    #Define tau2/tau1
    t21 = jett2/jett1
    
    #decorrelated tagger score
    t21dt = #Your decorrelated tagger score definition
    
    plt.hist2d(rho, t21dt, bins = 40)
    
    plt.xlabel(r"$\rho$")
    plt.ylabel(r"$\tau_2/\tau_1$_dt")
    plt.show()
    
plot_tausdt_and_rho(qcd)

Great! if you do it correctly, you can see that the decorrelated scores are now independent of $\rho$! Since you figured out your decorrelation, determine the best cut for the decorrelated taggers score (maybe by plotting the distribution between qcd background and the W signal like we did above) and use it in your selection function!

In [ ]:
def selectionW(iData):
    '''
    This is the specific selection for selecting out events with W signal for our analysis
    '''
    
    #Pre-selection citeria
    trigger = (iData.arrays('trigger', library="np")["trigger"].flatten() >= 0)
    jetpt   = (iData.arrays('vjet0_pt', library="np")["vjet0_pt"].flatten() >= 400)
    jetptnocut = (iData.arrays('vjet0_pt', library="np")["vjet0_pt"].flatten())
    jetmass = (iData.arrays('vjet0_msd0', library="np")["vjet0_msd0"].flatten())
    jett2   = (iData.arrays('vjet0_t2', library="np")["vjet0_t2"].flatten())
    jett1   = (iData.arrays('vjet0_t1', library="np")["vjet0_t1"].flatten())
    
    
    #Define the two parameters rho and tau2/tau1 
    rho = ### YOUR CODE HERE ###
    t21 = ### YOUR CODE HERE ###
    t21ddt = ### YOUR CODE HERE ###
                            
    #And then perform the cut
    #Hint: You could determine the threshold of the cut by plotting the distribution of 
    #t21ddt scores for W and background and then determine a ball park threshold
    #where you think the W signal would be best selected
    t21cut   = ### YOUR CODE HERE ###
    
    allcuts = np.logical_and.reduce([trigger, jetpt, t21cut])
    
    return allcuts

-----

Now that we have our selection function, let's try to make the mass plot!

In [ ]:
#Lets Look at the mass
weights = [1000*18300, "puweight", "scale1fb"]
mrange = (45,200)  #range for mass histogram [GeV]
bins=40            #bins for mass histogram
density = True     #to plot the histograms as a density (integral=1)

qcdsel      = selectionW(qcd)
wsel        = selectionW(wqq13)
zsel        = selectionW(zqq13)
datasel     = selectionW(data)
ttsel       = selectionW(tt)
wwsel       = selectionW(ww)
wzsel       = selectionW(wz)
zzsel       = selectionW(zz)
gghsel      = selectionW(ggh)
wscale=scale(wqq,wqq13,weights)
zscale=scale(zqq,zqq13,weights)

dataW = data.arrays('vjet0_msd0', library="np")["vjet0_msd0"][datasel]
qcdW = qcd.arrays('vjet0_msd0', library="np")["vjet0_msd0"][qcdsel]
wW = wqq13.arrays('vjet0_msd0', library="np")["vjet0_msd0"][wsel]
zW = zqq13.arrays('vjet0_msd0', library="np")["vjet0_msd0"][zsel]
zzW = zz.arrays('vjet0_msd0', library="np")["vjet0_msd0"][zzsel]
wzW = wz.arrays('vjet0_msd0', library="np")["vjet0_msd0"][wzsel]
wwW = ww.arrays('vjet0_msd0', library="np")["vjet0_msd0"][wwsel]
ttW = tt.arrays('vjet0_msd0', library="np")["vjet0_msd0"][ttsel]
gghW = ggh.arrays('vjet0_msd0', library="np")["vjet0_msd0"][gghsel]

hist_weights = [get_weights(qcd,weights,qcdsel),
                get_weights(wqq13,weights,wsel)*wscale,
                get_weights(zqq13,weights,zsel)*zscale,
                get_weights(zz,weights,zzsel),
                get_weights(wz,weights,wzsel),
                get_weights(ww,weights,wwsel),
                get_weights(tt,weights,ttsel),
               ]

plt.hist([qcdW,wW, zW, zzW, wzW, wwW, ttW], 
         color=["royalblue","r", "orange","g", "b", "purple", "cyan",], 
         label=["QCD", "W", "Z", "ZZ", "WZ", "WW", "tt",], 
         weights=hist_weights,
         range=mrange, bins=50, alpha=.6, density=density,stacked=True)

counts, bins = np.histogram(dataW, bins=bins, range=mrange, density=density)
yerr = np.sqrt(counts) / np.sqrt(len(dataW)*np.diff(bins))
binCenters = (bins[1:]+bins[:-1])*.5
plt.errorbar(binCenters, counts, yerr=yerr,fmt="o",c="k",label="data")
plt.legend()
plt.xlabel(r"Mass [GeV]")
plt.ylabel("Normalized Counts")
plt.show()

Remember to compare your mass plot with the one given above!

-------

## 1.2 W Peak Fit

In this section you will perform the fit on W signal. It involves a few steps:
    
1. First you need to define a fit model of your own. In this case we would use some functions (gaussian, exponential) in conjuntion with a 6th order polynomial. You could see more on how the order of the polynomials are determined here: https://en.wikipedia.org/wiki/Chow_test. The concepts were also covered in Lecture 8-9 if you want to review it. Later on you will have the chance to determine the order of the polynomial for the Z fit. You might see that we might not necessarily need a 6th order polynomial for the Z fit. The main reason for this is that we have much more data in W sample than the Z sample.

In [ ]:
def fitW():
    '''
    You should define other functions in conjunction with a 6th order polynomial
    '''
    
    return y

2. After defining your model, you need to get the data histogram and perform the fit:

In [ ]:
# Now we get the data histogram so we can fit it
bins = 50
mrange=[40,140]
counts, bins = np.histogram(dataW,bins=bins,range=mrange,density=False)

w = (1/ #poisson unc) #Poisson uncertainty here
binCenters = (bins[1:]+bins[:-1])*.5
x,y = binCenters.astype("float32"), counts.astype("float32")

#Perform the fit 
model = lm.Model(fitW)
p = model.make_params(#initial conditions for your fit) #You could experiment with zeros or your intuition first.

#For better fit I suggest adding restrictions to the fit.

result_W = model.fit(data=y,
                   params=p,
                   x=x,
                   weights=w)

#Plot the result
plt.figure()
result_W.plot()
plt.xlabel("mass[GeV]",position=(0.92,0.1))
plt.ylabel("Entries/bin",position=(0.1,0.84))

#Print the fit summary
print(result_W.fit_report())
result_W.chisqr

Remember to compare your fit plot! Now we need to extract the W mass and the error in the measurement!

In [ ]:
mW = result_W.params["mu"].value
mWerr = result_W.params["mu"].stderr

Your **W mass should be in the 80-90 GeV** range depending on the fit function of your choice!

----

# 2. Fit for Z peak

**NOTE: It is not mandatory that you completely finish this section before turning your work in!**

Now that you have done the W fit, the next challenge is to see whether you can also perform the Z fit! They are very similar in concepts. However, the selection for Z requires a special cut: **the deep double-b tagging score (ddb).** This is because unlike the W, Z can decays into two b quarks and hence a cut on this score would remove many W signals. 

The codes to perform the fit should be very similar to what we have shown you in the last section! Your plots should come out similarly to this in the end:

<img src="images/S50_ZFit.png" width='900'>

First, let's try to re-write the selection function. Remember that you now need to add the deep double b score of the jet and place a cut on it. The inividual b score in each subjet is already given in the `vjet0_sj1_csv` and `vjet0_sj2_csv` variable, how would you compute the b score of the whole jet?

Another hint for placing the cut is this following table: 

<img src="images/S50_ZBtagCut.png" width='500'>

You could just ignore the labelling for the most part, the main point here is that there are three "operating" or "working" points: tight, medium, loose. Each according to specific cuts on the **individual** subjet b score. For example, a mistag rate of 0.1 would require the b score to be more than 0.9. The tight working point is also what is generally used. Using this information could you pick an appropriate threshold for the whole jet?

An alternative approach to experimenting with different working points is for you to plot the distribution of the b scores of the Z signal versus W and QCD like we did in Section 0. From the histogram we could determine the appropriate cut.

In [ ]:
# Let's define a selection that will keep Z jets and remove background.
def selectionZ(iData):
    
        #Standard preselection
        trigger = (iData.arrays('trigger', library="np")["trigger"].flatten() >= 0)
        
        #Now lets require the jet pt to be above a threshold
        jetpt   = (iData.arrays('vjet0_pt', library="np")["vjet0_pt"].flatten() >= 400)
        jetmass = (iData.arrays('vjet0_msd0', library="np")["vjet0_msd0"].flatten())
        jett2   = (iData.arrays('vjet0_t2', library="np")["vjet0_t2"].flatten())
        jett1   = (iData.arrays('vjet0_t1', library="np")["vjet0_t1"].flatten())
        jetptnocut = (iData.arrays('vjet0_pt', library="np")["vjet0_pt"].flatten())
        jetcsvsj1  = (iData.arrays('vjet0_sj1_csv', library="np")["vjet0_sj1_csv"].flatten())
        jetcsvsj2  = (iData.arrays('vjet0_sj2_csv', library="np")["vjet0_sj2_csv"].flatten())
        
        # Following the same procedure as https://arxiv.org/pdf/1603.00027.pdf
        # we select 2 progned jets (since W and Z primarily decay to 2 quarks)
        # we correct t21 to minimize the dependance on m and pt
        rho = ### YOUR CODE HERE ###
        t21 = ### YOUR CODE HERE ###
        
        t21ddt = ### YOUR CODE HERE ###
        t21cut = ### YOUR CODE HERE ### #You could use a similar cut to W, but looser
        
        # this cut keeps jets with subjet 1 and subjet 2 coming from a secondary vertex
        # i.e. the subjets originate from a displaced particle (namely a b quark)
        # Unlike W bosons, the Z can decay into 2 b quarks which is why this selection removes 
        # many W jets.
    
        ddb = ### YOUR CODE HERE ###
        ddbcut = ### YOUR CODE HERE ###
        
        allcuts = np.logical_and.reduce([trigger,jetpt, t21cut, ddbcut])
        
        return allcuts

Similar to the W, let's try to make the mass plot first to see if we get the correct mass plot:

## 2.1 Z mass plot

We can basically just reused what we had in the last section

In [ ]:
# Some default parameters
weights=[1000*18300,"puweight","scale1fb"]
mrange = (50,200) #range for mass histogram [GeV]
bins=50           #bins for mass histogram
density = True    #to plot the histograms as a density (integral=1)

# Selecting data and MC 
qcdsel      = selectionZ(qcd)
wsel        = selectionZ(wqq13)
zsel        = selectionZ(zqq13)
datasel     = selectionZ(data)
ttsel       = selectionZ(tt)
wwsel       = selectionZ(ww)
wzsel       = selectionZ(wz)
zzsel       = selectionZ(zz)
gghsel      = selectionZ(ggh)

wscale=scale(wqq,wqq13,weights)
zscale=scale(zqq,zqq13,weights)

# Getting the masses of selected events
datat21 = data.arrays('vjet0_msd0', library="np")["vjet0_msd0"][datasel]
qcdt21 = qcd.arrays('vjet0_msd0', library="np")["vjet0_msd0"][qcdsel]
wt21 = wqq13.arrays('vjet0_msd0', library="np")["vjet0_msd0"][wsel]
zt21 = zqq13.arrays('vjet0_msd0', library="np")["vjet0_msd0"][zsel]
zzt21 = zz.arrays('vjet0_msd0', library="np")["vjet0_msd0"][zzsel]
wzt21 = wz.arrays('vjet0_msd0', library="np")["vjet0_msd0"][wzsel]
wwt21 = ww.arrays('vjet0_msd0', library="np")["vjet0_msd0"][wwsel]
ttt21 = tt.arrays('vjet0_msd0', library="np")["vjet0_msd0"][ttsel]
gght21 = ggh.arrays('vjet0_msd0', library="np")["vjet0_msd0"][gghsel]

# Proper weights for each histogram
hist_weights = [get_weights(qcd,weights,qcdsel),
                get_weights(wqq13,weights,wsel)*wscale,
                get_weights(zqq13,weights,zsel)*zscale,
                get_weights(zz,weights,zzsel),
                get_weights(wz,weights,wzsel),
                get_weights(ww,weights,wwsel),
                get_weights(tt,weights,ttsel),
               ]

# Plot stacked histograms
plt.hist([qcdt21,wt21, zt21, zzt21, wzt21, wwt21, ttt21], 
         color=["royalblue","r", "orange","g", "b", "purple", "cyan"], 
         label=["QCD", "W", "Z", "ZZ", "WZ", "WW", "tt"], 
         weights=hist_weights,
         range=mrange, bins=50, alpha=.6, density=density,stacked=True)
counts, bins = np.histogram(datat21,bins=bins,range=mrange,density=density)

# Getting the proper err requires normalizing since we are using a density not the actual count
yerr = np.sqrt(counts) / np.sqrt(len(datat21)*(np.diff(bins))) 
binCenters = (bins[1:]+bins[:-1])*.5
plt.errorbar(binCenters, counts, yerr=yerr,fmt="o",c="k",label="data")
plt.legend()
plt.xlabel(r"Mass [GeV]")
plt.ylabel("Normalized Counts")
plt.show()

Make sure to compare your mass plot with the plot given above!

## 2.1 Fit for Z peak

Now, adapting the codes provided for you in the W section, perform a fit to the Z peak and extract its mass! The concept is similar to our previous W fit, but this time I would let you explore what fit function would be appropriate in this case.

In [ ]:
def fitZ(x,):
    
    return 

----

Then, get the data histogram and perform the fit

In [ ]:
# Now we get the data histogram so we can fit it
counts, bins = np.histogram(datat21,
                            bins=40,
                            range=mrange)

w = 1/np.sqrt(counts)
binCenters = (bins[1:]+bins[:-1])*.5
x,y = binCenters.astype("float32"), counts.astype("float32")

#Perform the fit
model = lm.Model(fitZ)
p = model.make_params(p0=10,
                      p1=50,
                      p2=0,
                      p3=50,
                      p4=0,
                      p5=0,
                      a=0,
                      mu=90,
                      sigma=5)

result_Z = model.fit(data=y,
                     params=p,
                     x=x,
                     weights=w)

#Plottting
plt.figure()
result_Z.plot()
plt.xlabel("mass[GeV]", position=(0.92,0.1))
plt.ylabel("Entries/bin", position=(0.1,0.84))

#We could also check out all the fit parameters
print(result_Z.fit_report())

After the fit we can extract the Z mass and the error in our measurement

In [ ]:
mZ = result_Z.params["mu"].value
mZerr = result_Z.params["mu"].stderr

Note that the Z mass should be around **91 GeV**.

-----

# 3. Weak mixing angle (OPTIONAL)

If you have both W and Z boson, you can measure a fundamental parameter of the standard model. The Weak mixing angle, sometimes referred to as the Weinberg angle. 

<font color="blue"> From your measurement of the W and Z boson mass come up with a measurement of the W and Z boson mixing angle.  </font>

we have that $\sin^2\theta_w = 1 - (\frac{m_W}{m_Z})^2$ which was measured to be about 0.231. Using the masses we measured and propagating the error gives 

In [ ]:
sin2theta = 1 - (mW/mZ)**2
sin2theta_err = ### YOUR CODE HERE ###

print(f"sin^2(θ) = {sin2theta:.3f} +\- {sin2theta_err:.3f}")

Hmm... It looks like our measurement is pretty bad.  The reason for this is that we overestimated the mass of the W boson because many of the "signal" events selected contain a Z boson which has higher mass. Because we can't easily find a selection that removes Z jets and keeps W jets we will resort to making a correction from the Monte Carlo (MC). Since we have control over what samples go into the MC we can actually do our fit with and without Z jets to see how biased our procedure is. Then we can correct for this bias approriately.

-----

# 4. Correcting $m_W$   ( OPTIONAL)

First we need to perform the selection for W again

In [ ]:
#Lets Look at the mass
weights = [1000*18300, "puweight", "scale1fb"]
mrange = (45,200)  #range for mass histogram [GeV]
bins=40            #bins for mass histogram
density = True     #to plot the histograms as a density (integral=1)

qcdsel      = selectionW(qcd)
wsel        = selectionW(wqq13)
zsel        = selectionW(zqq13)
datasel     = selectionW(data)
ttsel       = selectionW(tt)
wwsel       = selectionW(ww)
wzsel       = selectionW(wz)
zzsel       = selectionW(zz)
gghsel      = selectionW(ggh)
wscale=scale(wqq,wqq13,weights)
zscale=scale(zqq,zqq13,weights)

dataW = data.arrays('vjet0_msd0', library="np")["vjet0_msd0"][datasel]
qcdW = qcd.arrays('vjet0_msd0', library="np")["vjet0_msd0"][qcdsel]
wW = wqq13.arrays('vjet0_msd0', library="np")["vjet0_msd0"][wsel]
zW = zqq13.arrays('vjet0_msd0', library="np")["vjet0_msd0"][zsel]
zzW = zz.arrays('vjet0_msd0', library="np")["vjet0_msd0"][zzsel]
wzW = wz.arrays('vjet0_msd0', library="np")["vjet0_msd0"][wzsel]
wwW = ww.arrays('vjet0_msd0', library="np")["vjet0_msd0"][wwsel]
ttW = tt.arrays('vjet0_msd0', library="np")["vjet0_msd0"][ttsel]
gghW = ggh.arrays('vjet0_msd0', library="np")["vjet0_msd0"][gghsel]

After that we could evaluate the MC simulation in two cases:

1. MC with Z
2. MC without Z

And then based on the fit of these two samples, we could evaluate the ratio of the fitted W mass and determine the bias factor, which can then be used to correct the W mass!

In this lab we can perform a simple procedure to determine the bias by plotting the MC distribution of the W mass with selection. After that, we could take the mean and standard deviation with and without the Z to feed into our bias calculation! A more advanced approach would be to perform the whole W fit again with QCD MC. However, the fit is rather hard to control (getting a good $\chi^2$ is hard). Therefore, for the purposes of this course, we would resort to a simpler verion first!

In [ ]:
bins=50

# We need the weights from our MC
wweights = get_weights(wqq13,weights,wsel)*wscale
zweights = get_weights(zqq13,weights,zsel)*zscale

mrange = [50,150]

# Now we stack the histograms into a single density 
counts_mc, bins,_ = plt.hist([wW,zW],bins=bins,range=mrange,
                             density=False,stacked=True,
                             color=["r", "orange"],
                             alpha=.6,
                             weights= [wweights, zweights])


binCenters = (bins[1:]+bins[:-1])*.5
plt.show()

mW_mc_withZ = np.average(binCenters, weights=counts_mc[-1])
mWerr_mc_withZ = ### YOUR CODE HERE ###

mW_mc_noZ = np.average(binCenters,weights=counts_mc[0])
mWerr_mc_noZ = ### YOUR CODE HERE ###

print("W mass with Z: {}. W mass without Z: {}".format(mW_mc_withZ, mW_mc_noZ))
print("W mass (standard deviation) with Z: {}. W mass (standard deviation) without Z: {}".format(mWerr_mc_withZ, mWerr_mc_noZ))

Then we can obtain the mass with no Z in the sample:

## 4.3 Bias factor and correct for W mass

Now that we obtain the W mass in both cases, let's correct for the W mass!

In [ ]:
# now we calculate our bias factor
# if the W mass measured with and without Z is the same 
# this factor should be 1
bias = mW_mc_withZ / mW_mc_noZ

#propagate the error assuming uncorrelated gaussian uncertainties
bias_err = ### YOUR CODE HERE ###

-----

In [ ]:
mW_corrected = mW/bias
mWerr_corrected = ### YOUR CODE HERE ###

-----

In [ ]:
print(f"Correcting the W mass using MC (simulated data) gives m_W = {mW_corrected:.1f} +\- {mWerr_corrected:.1f}")

In [ ]:
sin2theta = 1 - (mW_corrected/mZ)**2
sin2theta_err = ### YOUR CODE HERE ###
print(f"sin^2(θ) = {sin2theta:.3f} +\- {sin2theta_err:.3f}")

-----

And the new $Sin^2(\theta)$ should be at around:

$$Sin^2(\theta) = 0.21$$

This measurement is much better than the previous one! Congratulations on making it this far to this lab!

Now if you have made it through this whole exercise, why don't you try to look for the Higgs boson. The Higgs boson also decays hadronically and decays predominantly to a certainy pair of particles. Take a look [here](http://pdg.lbl.gov/2019/reviews/rpp2019-rev-higgs-boson.pdf). 
 <font color="blue"> Can you search for the Higgs boson? What is your sensitivity.  </font>

------

## 4. Search for the Higgs Boson! (OPTIONAL)